In [1]:
import json
import struct
import numpy as np
import matplotlib.pyplot as plt
import photonforge as pf
import siepic_forge as siepic
import luxtelligence_lnoi400_forge as lxt
import tidy3d as td

td.config.logging_level = "ERROR"

# Set up technologies
siepic_tech = siepic.ebeam()
basic_tech = pf.basic_technology()
lxt_tech = lxt.lnoi400()
pf.config.default_technology = siepic_tech

# Initialize live viewer for real-time visualization
from photonforge.live_viewer import LiveViewer
viewer = LiveViewer(5005)

# Define simulation parameters
wavelengths = np.linspace(1.53, 1.57, 101)
freqs = pf.C_0 / wavelengths

LiveViewer started at http://localhost:5005


In [2]:
class ThermalModel(pf.Model):
    def __init__(self, n_complex, voltage=0, coefficient=3e-4):
        super().__init__(
            n_complex=n_complex,
            voltage=voltage,
            coefficient=coefficient,
        )
        self.n_complex = np.array(n_complex, ndmin=2)
        self.voltage = voltage
        self.coefficient = coefficient

    def __copy__(self):
        return ThermalModel(self.n_complex, self.voltage, self.coefficient)

    def __deepcopy__(self, memo=None):
        # n_complex is an array, so we want to make sure to create a deep copy of it.
        # Other values (voltage and coefficient) are immutable (floats), so we can use them directly.
        return ThermalModel(self.n_complex.copy(), self.voltage, self.coefficient)

    def __repr__(self):
        return f"ThermalModel({self.n_complex!r}, {self.voltage!r}, {self.coefficient!r})"

    def __str__(self):
        return f"ThermalModel at {self.voltage} V"

    @property
    def as_bytes(self):
        coeffs = struct.pack("<2d", self.voltage, self.coefficient)
        shape = struct.pack("<2l", *self.n_complex.shape)
        n_data = self.n_complex.astype(complex).tobytes()
        # Add version 0 as first byte
        return b"\x00" + coeffs + shape + n_data

    @classmethod
    def from_bytes(cls, byte_repr):
        version = byte_repr[0]
        if version != 0:
            raise RuntimeError(f"Incompatible version for ThermalModel: {version}")

        byte_repr = byte_repr[1:]
        fmt = "<2d2l"
        head_len = struct.calcsize(fmt)
        voltage, coefficient, rows, cols = struct.unpack(fmt, byte_repr[:head_len])

        byte_repr = byte_repr[head_len:]
        n_complex = np.frombuffer(byte_repr, dtype=complex).reshape((rows, cols))

        return cls(n_complex, voltage, coefficient)

    @pf.cache_s_matrix
    def start(self, component, frequencies, voltage=None, **kwargs):
        # Allow overriding voltage as an `s_matrix` kwarg too
        if voltage is None:
            voltage = self.voltage
        n_complex = self.n_complex + self.coefficient * voltage**2
        wg_model = pf.WaveguideModel(n_complex)
        return wg_model.start(component, frequencies, **kwargs)


pf.register_model_class(ThermalModel)

In [11]:
@pf.parametric_component
def create_tunable_mzi_siepic(*, name, coupling_distance=0.6, coupling_length=5.35, ps_length=5):

    # Bend and Straight waveguide to form the microring
    phase_shifter = pf.parametric.straight(name=f"ps", port_spec="Rib_TE_1550_500", length=ps_length)
    straight = pf.parametric.straight(port_spec="TE_1550_500", length=ps_length+10)
    bend = pf.parametric.bend(port_spec="TE_1550_500", radius=5)
    trans = pf.parametric.transition(port_spec1="TE_1550_500", port_spec2="Rib_TE_1550_500", length=5)
    coupler = pf.parametric.dual_ring_coupler(
        port_spec="TE_1550_500", coupling_distance=coupling_distance, coupling_length=coupling_length, radius=5, 
        tidy3d_model_kwargs={
            # These symmetries are not correct for arbitrary ports.
            # They can be used in this case for the fundamental mode.
            "port_symmetries": [
                ("P1", "P0", "P3", "P2"),
                ("P2", "P3", "P0", "P1"),
                ("P3", "P2", "P1", "P0"),
            ],
        },
    )

    # Solve for the port mode of the waveguide and extract the complex refractive index
    alpha = 10
    kappa = (alpha * wavelengths * 1e-4 * np.log(10)) / (40 * np.pi)
    mode_solver = pf.port_modes(port=phase_shifter.ports["P0"], frequencies=freqs)
    n_complex = mode_solver.data.n_complex.values.T + 1j * kappa  # add propagation loss

    thermal_model = ThermalModel(n_complex=n_complex)
    phase_shifter.add_model(thermal_model, "Thermal")

    tunable_mzi = pf.Component(name)

    # form main microring structure
    cp1_ref = tunable_mzi.add_reference(coupler)
    cp2_ref = tunable_mzi.add_reference(coupler)
    ps1_ref = tunable_mzi.add_reference(phase_shifter)
    bend1_ref = tunable_mzi.add_reference(bend)
    bend2_ref = tunable_mzi.add_reference(bend) 
    bend3_ref = tunable_mzi.add_reference(bend)
    bend4_ref = tunable_mzi.add_reference(bend) 
    str1_ref = tunable_mzi.add_reference(straight)
    trans1_ref = tunable_mzi.add_reference(trans)
    trans2_ref = tunable_mzi.add_reference(trans)

    #Coupler Kiri
    bend1_ref.connect("P1", cp1_ref["P3"])
    bend2_ref.connect("P0", cp1_ref["P2"])
    #Tengah
    trans1_ref.connect("P0", bend1_ref["P0"])
    ps1_ref.connect("P0", trans1_ref["P1"])
    str1_ref.connect("P0", bend2_ref["P1"])

    trans2_ref.connect("P1", ps1_ref["P1"])
    bend3_ref.connect("P1", trans2_ref["P0"])
    bend4_ref.connect("P0", str1_ref["P1"])
    #Coupler Kanan
    cp2_ref.connect("P1", bend3_ref["P0"])
    cp2_ref.connect("P0", bend4_ref["P1"])


    # Define the heater and electrical path
    terminal_width = 10
    heater_width = 1

    heater = (
        pf.Path((ps1_ref.x_min, ps1_ref.y_mid), heater_width)
        .segment((ps1_ref.x_max, ps1_ref.y_mid), heater_width)
    )

    route_vp = (
        pf.Path((cp1_ref.x_mid-terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((cp1_ref.x_mid+terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((ps1_ref.x_min, ps1_ref.y_mid), heater_width)
    )

    route_vn = (
        pf.Path((ps1_ref.x_max, ps1_ref.y_mid), heater_width)
        .segment((cp2_ref.x_mid-terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((cp2_ref.x_mid+terminal_width/2, ps1_ref.y_mid), terminal_width)
    )

    tunable_mzi.add((11,0), heater)
    tunable_mzi.add((12,0), route_vp)
    tunable_mzi.add((12,0), route_vn)
    tunable_mzi.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), center=(cp1_ref.x_mid, ps1_ref.y_mid))), "VP")
    tunable_mzi.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), center=(cp2_ref.x_mid, ps1_ref.y_mid))), "VN")

    ## COMPONENT MODELS
    tunable_mzi.add_port(tunable_mzi.detect_ports(["TE_1550_500"]))
    tunable_mzi.add_model(pf.CircuitModel(), "CircuitModel")

    port_symmetries = [
        ("P1", "P0", "P3", "P2"),
        ("P2", "P3", "P0", "P1"),
        ("P3", "P2", "P1", "P0"),
    ]

    field_monitor = td.FieldMonitor(
        center=(0, 0, 0.11), size=(td.inf, td.inf, 0), freqs=[freqs.mean()], name="field"
    )

    tunable_mzi.add_model(pf.Tidy3DModel(port_symmetries=port_symmetries, monitors=[field_monitor]), "Tidy3DModel")

    return tunable_mzi

# tunable_mzi = create_tunable_mzi_siepic(name="mzm", ps_length=99.3)
# viewer(tunable_mzi)

c:\Users\irfan\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:208: RuntimeWarning: Component function '__main__.create_tunable_mzi_siepic' previously registered will be overwritten.
  return _decorator(decorated_function)


In [37]:
alice_top = pf.Component("alice_top")
# ring_resonator = parametric_ring_resonator(port_spec="Rib_TE_1550_500", radius=10, bus_length=15, coupling_distance=0.6)
# tunable_resonator = create_tunable_resonator(port_spec="Rib_TE_1550_500", radius=7, coupling_distance=0.6, ps_length=6)
straight = pf.parametric.straight(port_spec="TE_1550_500", length=140)
half_straight = pf.parametric.straight(port_spec="TE_1550_500", length=50)
quarter_straight = pf.parametric.straight(port_spec="TE_1550_500", length=10)
y_branch = siepic.component("ebeam_y_1550")
grating_coupler = siepic.component("ebeam_gc_te1550")
bend = pf.parametric.bend(port_spec="TE_1550_500", radius=5, euler_fraction=0.5, angle=90)
sbend_top = pf.parametric.s_bend(port_spec="TE_1550_500", length=30, offset=-10.85, euler_fraction=0.5)
sbend_bottom = pf.parametric.s_bend(port_spec="TE_1550_500", length=30, offset=10.85, euler_fraction=0.5)
wider_sbend_top = pf.parametric.s_bend(port_spec="TE_1550_500", length=45, offset=-24.45, euler_fraction=0.5)
wider_sbend_bottom = pf.parametric.s_bend(port_spec="TE_1550_500", length=45, offset=24.45, euler_fraction=0.5)
trans = pf.parametric.transition(port_spec1="TE_1550_500", port_spec2="Rib_TE_1550_500", length=5, constant_length=0.5)
# tunable_mzi = create_tunable_mzi(port_spec="Rib_TE_1550_500", coupling_distance=0.6, ps_length=10)
mzm_base = create_tunable_mzi_siepic(name="mzm_base", ps_length=99.3)
mzm_bit = create_tunable_mzi_siepic(name="mzm_bit", ps_length=99.3)

gc1_ref = alice_top.add_reference(grating_coupler)
gc2_ref = alice_top.add_reference(grating_coupler)

# rr1_ref = alice_top.add_reference(ring_resonator)
# rr2_ref = alice_top.add_reference(ring_resonator)
# rr3_ref = alice_top.add_reference(ring_resonator)

MZI1_ref = alice_top.add_reference(mzm_base)
MZI2_ref = alice_top.add_reference(mzm_bit)
MZI3_ref = alice_top.add_reference(mzm_bit)
MZI4_ref = alice_top.add_reference(mzm_bit)
MZI5_ref = alice_top.add_reference(mzm_bit)
MZI6_ref = alice_top.add_reference(mzm_base)

st1_ref = alice_top.add_reference(straight)
st2_ref = alice_top.add_reference(straight)
st3_ref = alice_top.add_reference(straight)
st4_ref = alice_top.add_reference(straight)
# st5_ref = alice_top.add_reference(straight)
# st6_ref = alice_top.add_reference(straight)
# st7_ref = alice_top.add_reference(straight)
# st8_ref = alice_top.add_reference(straight)

hst1_ref = alice_top.add_reference(half_straight)
hst2_ref = alice_top.add_reference(half_straight)
hst3_ref = alice_top.add_reference(half_straight)
hst4_ref = alice_top.add_reference(half_straight)
hst5_ref = alice_top.add_reference(half_straight)
hst6_ref = alice_top.add_reference(half_straight)
# hst7_ref = alice_top.add_reference(half_straight)
# hst8_ref = alice_top.add_reference(half_straight)
# hst9_ref = alice_top.add_reference(half_straight)   

qst1_ref = alice_top.add_reference(quarter_straight)
qst2_ref = alice_top.add_reference(quarter_straight)
qst3_ref = alice_top.add_reference(quarter_straight)
qst4_ref = alice_top.add_reference(quarter_straight)
qst5_ref = alice_top.add_reference(quarter_straight)
qst6_ref = alice_top.add_reference(quarter_straight)
qst7_ref = alice_top.add_reference(quarter_straight)
qst8_ref = alice_top.add_reference(quarter_straight)
qst9_ref = alice_top.add_reference(quarter_straight)
qst10_ref = alice_top.add_reference(quarter_straight)
# qst11_ref = alice_top.add_reference(quarter_straight)
# qst12_ref = alice_top.add_reference(quarter_straight)
# qst13_ref = alice_top.add_reference(quarter_straight)

bend1_ref = alice_top.add_reference(bend)
bend2_ref = alice_top.add_reference(bend)
bend3_ref = alice_top.add_reference(bend)
bend4_ref = alice_top.add_reference(bend)
bend5_ref = alice_top.add_reference(bend)
bend6_ref = alice_top.add_reference(bend)
bend7_ref = alice_top.add_reference(bend)
bend8_ref = alice_top.add_reference(bend)   
bend9_ref = alice_top.add_reference(bend)
bend10_ref = alice_top.add_reference(bend)
bend11_ref = alice_top.add_reference(bend)
bend12_ref = alice_top.add_reference(bend)
bend13_ref = alice_top.add_reference(bend)
bend14_ref = alice_top.add_reference(bend)
bend15_ref = alice_top.add_reference(bend)
bend16_ref = alice_top.add_reference(bend)
bend17_ref = alice_top.add_reference(bend)
bend18_ref = alice_top.add_reference(bend)
# bend19_ref = alice_top.add_reference(bend)

# sbt1_ref = alice_top.add_reference(sbend_top)
# sbt2_ref = alice_top.add_reference(sbend_top)
# sbt3_ref = alice_top.add_reference(sbend_top)

# sbb1_ref = alice_top.add_reference(sbend_bottom)
# sbb2_ref = alice_top.add_reference(sbend_bottom)
# sbb3_ref = alice_top.add_reference(sbend_bottom)


# y1_ref = alice_top.add_reference(y_branch)
# y2_ref = alice_top.add_reference(y_branch)
# y3_ref = alice_top.add_reference(y_branch)

# trans1_ref = alice_top.add_reference(trans)
# trans2_ref = alice_top.add_reference(trans)
# trans3_ref = alice_top.add_reference(trans)
# trans4_ref = alice_top.add_reference(trans)
# trans5_ref = alice_top.add_reference(trans)
# trans6_ref = alice_top.add_reference(trans)
# trans7_ref = alice_top.add_reference(trans)
# trans8_ref = alice_top.add_reference(trans)
# trans9_ref = alice_top.add_reference(trans)
# trans10_ref = alice_top.add_reference(trans)

#Grating Coupler Input, MZI 1
hst1_ref.connect("P0",gc1_ref["P0"])
bend1_ref.connect("P1",hst1_ref["P1"])
# trans1_ref.connect("P0",bend1_ref["P0"])
MZI1_ref.connect("P1",bend1_ref["P0"])
# trans2_ref.connect("P1",MZI1_ref["P3"])

#MZI 2 (atas), Rotator 90, Y Branch 1
# trans3_ref.connect("P0",trans2_ref["P0"])
qst5_ref.connect("P0",MZI1_ref["P3"])
bend6_ref.connect("P0",qst5_ref["P1"])
st1_ref.connect("P1",bend6_ref["P1"])
bend7_ref.connect("P1",st1_ref["P0"]) 
qst6_ref.connect("P0",bend7_ref["P0"])
MZI2_ref.connect("P0",qst6_ref["P1"])
# trans4_ref.connect("P1",MZI2_ref["P3"])
bend2_ref.connect("P1",MZI2_ref["P3"])
hst3_ref.connect("P0",bend2_ref["P0"])
bend12_ref.connect("P1",hst3_ref["P1"])

# trans5_ref.connect("P1",MZI2_ref["P2"])
bend3_ref.connect("P0",MZI2_ref["P2"])
hst4_ref.connect("P0",bend3_ref["P1"])
bend4_ref.connect("P0",hst4_ref["P1"])

#MZM bit pengganti y-branch
MZI4_ref.connect("P0",bend4_ref["P1"])
qst1_ref.connect("P0",MZI4_ref["P2"])
bend14_ref.connect("P1",qst1_ref["P1"])
st3_ref.connect("P1",bend14_ref["P0"])
bend15_ref.connect("P0",st3_ref["P0"])
qst2_ref.connect("P1",bend15_ref["P1"])
# bend18_ref.connect("P0",qst2_ref["P0"])

#Rotate, Y Branch
# y1_ref.rotate(180)

# y1_ref.x_min = qst2_ref.x_max + 10
# y1_ref.y_mid = (qst2_ref.y_mid + qst1_ref.y_mid) / 2

# route_y1_1 = pf.parametric.route(port1=(qst1_ref, "P1"), port2=(y1_ref, "P1"), radius=5)
# route_y1_2 = pf.parametric.route(port1=(qst2_ref, "P1"), port2=(y1_ref, "P2"), radius=5)
# alice_top.add_reference(route_y1_1)
# alice_top.add_reference(route_y1_2)

#MZI 3 (bawah), Rotator 45, -45, Y Branch 2
# trans6_ref.connect("P1",MZI1_ref["P2"])
# trans7_ref.connect("P0",trans6_ref["P0"])
qst7_ref.connect("P0",MZI1_ref["P2"])
bend8_ref.connect("P1",qst7_ref["P1"])
st2_ref.connect("P1",bend8_ref["P0"])
bend9_ref.connect("P0",st2_ref["P0"])
qst8_ref.connect("P1",bend9_ref["P1"])
MZI3_ref.connect("P1",qst8_ref["P0"])
# trans8_ref.connect("P1",MZI3_ref["P3"])
bend10_ref.connect("P1",MZI3_ref["P3"])
hst5_ref.connect("P0",bend10_ref["P0"])
bend13_ref.connect("P1",hst5_ref["P1"])

# trans9_ref.connect("P1",MZI3_ref["P2"])
bend11_ref.connect("P0",MZI3_ref["P2"])
hst6_ref.connect("P0",bend11_ref["P1"])
bend5_ref.connect("P0",hst6_ref["P1"])

#MZM bit pengganti y-branch
MZI5_ref.connect("P0",bend5_ref["P1"])
qst9_ref.connect("P0",MZI5_ref["P3"])
bend16_ref.connect("P0",qst9_ref["P1"])
st4_ref.connect("P1",bend16_ref["P1"])
bend17_ref.connect("P1",st4_ref["P0"])
qst10_ref.connect("P1",bend17_ref["P0"])
# bend19_ref.connect("P1",qst10_ref["P0"])

#Rotate, Y Branch
# y2_ref.rotate(180)

# y2_ref.x_min = qst10_ref.x_max + 10
# y2_ref.y_mid = (qst10_ref.y_mid + qst9_ref.y_mid) / 2

# route_y2_1 = pf.parametric.route(port1=(qst9_ref, "P1"), port2=(y2_ref, "P1"), radius=5)
# route_y2_2 = pf.parametric.route(port1=(qst10_ref, "P1"), port2=(y2_ref, "P2"), radius=5)
# alice_top.add_reference(route_y2_1)
# alice_top.add_reference(route_y2_2)

#Y Branch, Grating Coupler Output
MZI6_ref.connect("P0",qst10_ref["P0"])
bend18_ref.connect("P1",MZI6_ref["P3"])
# y3_ref.rotate(180)

# y3_ref.x_min = y2_ref.x_max + 10
# y3_ref.y_mid = (y2_ref.y_mid + y1_ref.y_mid) / 2

# route_y3_1 = pf.parametric.route(port1=(y1_ref, "P0"), port2=(y3_ref, "P1"), radius=5)
# route_y3_2 = pf.parametric.route(port1=(y2_ref, "P0"), port2=(y3_ref, "P2"), radius=5)
# alice_top.add_reference(route_y3_1)
# alice_top.add_reference(route_y3_2)

hst2_ref.connect("P0",bend18_ref["P0"])
gc2_ref.connect("P0",hst2_ref["P1"])

#ELECTRICAL CONNECTIONS
terminal_width = 15

# Terminals
alice_top.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), 
                                                        center=(MZI2_ref.x_min + terminal_width/2 + 0.425, MZI2_ref.y_max + terminal_width/2 + 10))), 
                       "GND1", add_structure=True)

alice_top.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), 
                                                        center=(MZI4_ref.x_max - terminal_width/2 - 0.425, MZI2_ref.y_max + terminal_width/2 + 10))), 
                       "GND2", add_structure=True)

alice_top.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), 
                                                        center=(hst3_ref.x_mid, MZI2_ref.y_max + terminal_width/2 + 10))), 
                       "V_BIT", add_structure=True)

alice_top.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), 
                                                        center=(hst6_ref.x_mid, hst6_ref.y_min - terminal_width/2 - 15))), 
                       "V_BASE", add_structure=True)

pads = alice_top.terminals

# Electrical routing

# V_BIT
alice_top.add(
    pf.parametric.route_manhattan(
        terminal1=pads["V_BIT"],
        terminal2=(MZI2_ref, "VN", 0),
        direction1="y",
        direction2="y",
        width=10
    ),
    pf.parametric.route_manhattan(
        terminal1=pads["V_BIT"],
        terminal2=(MZI3_ref, "VN", 0),
        direction1="y",
        direction2="y",
        width=10
    ),
    pf.parametric.route_manhattan(
        terminal1=pads["V_BIT"],
        terminal2=(MZI4_ref, "VP", 0),
        direction1="y",
        direction2="y",
        width=10
    ),
    pf.parametric.route_manhattan(
        terminal1=pads["V_BIT"],
        terminal2=(MZI5_ref, "VP", 0),
        direction1="y",
        direction2="y",
        width=10
    ),
)

# V_BASE
alice_top.add(
    pf.parametric.route_manhattan(
        terminal1=(MZI1_ref, "VP", 0),
        terminal2=pads["V_BASE"],
        direction1="x",
        direction2="y",
        width=10,
        waypoints=[(MZI1_ref.x_min + terminal_width/2 + 0.425 - 20, MZI1_ref.y_max), 
                   (MZI1_ref.x_min + terminal_width/2 + 0.425 - 20, MZI3_ref.y_min),
                   (hst6_ref.x_mid, hst6_ref.y_min),
                   ]
    ),
    pf.parametric.route_manhattan(
        terminal1=(MZI6_ref, "VN", 0),
        terminal2=pads["V_BASE"],
        direction1="x",
        direction2="y",
        width=10,
        waypoints=[(MZI6_ref.x_max - terminal_width/2 - 0.425 + 20, MZI6_ref.y_max), 
                   (MZI6_ref.x_max - terminal_width/2 - 0.425 + 20, MZI5_ref.y_min),
                   (hst6_ref.x_mid, hst6_ref.y_min),
                   ]
    )
)

# GND
alice_top.add(
    pf.parametric.route_manhattan(
        terminal1=pads["GND1"],
        terminal2=(MZI3_ref, "VP", 0),
        direction1="y",
        direction2="y",
        width=10,
        waypoints=[(MZI2_ref.x_min + terminal_width/2 + 0.425, MZI2_ref.y_min), 
                   (MZI2_ref.x_max - terminal_width/2 - 0.425, MZI2_ref.y_min),
                   (MZI1_ref.x_max - terminal_width/2 - 0.425, MZI1_ref.y_min),
                   (MZI1_ref.x_min + terminal_width/2 + 0.425, MZI1_ref.y_min), 
                   ]
    ),
    pf.parametric.route_manhattan(
        terminal1=pads["GND2"],
        terminal2=(MZI5_ref, "VN", 0),
        direction1="y",
        direction2="y",
        width=10,
        waypoints=[(MZI4_ref.x_max - terminal_width/2 - 0.425, MZI4_ref.y_min), 
                   (MZI4_ref.x_min + terminal_width/2 + 0.425, MZI4_ref.y_min),
                   (MZI6_ref.x_min + terminal_width/2 + 0.425, MZI6_ref.y_min),
                   (MZI6_ref.x_max - terminal_width/2 - 0.425, MZI6_ref.y_min), 
                   ]
    ),
)

viewer(alice_top)

Progress: 100% 
Progress: 100% 


c:\Users\irfan\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:168: RuntimeWarning: Skipped point (42.675, 5) from route
  c = component_func(*args, **kwargs)
c:\Users\irfan\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:168: RuntimeWarning: Skipped point (427.325, 5) from route
  c = component_func(*args, **kwargs)
